In [6]:
import os
import pandas as pd
import numpy as np
import sys
import operator

In [7]:
input_directory = os.path.join('..', '..', 'data', 'input_files_multimodal_SR41', 'single_class_input_files')
output_directory = os.path.join('..', '..', 'data', 'input_files_multimodal_SR41')

## Read transit data

In [18]:
input_file_name_transit = os.path.join(output_directory, 'transit_route_stops')
bus_route_dic = {} # route_name : {'frequency(s)': int, 'DMOND_ID': str, 'DMDND_ID': str, 'stops':[list of node ID]}
with open(input_file_name_transit, 'r') as infile:
    for line in infile.readlines():
        if line[0] != '#':
            data = line.rstrip('\n').split(' ')
#             print data
            bus_route_dic[data[0]] = {}
            bus_route_dic[data[0]]['frequency'] =  int(data[1])
            bus_route_dic[data[0]]['DMOND_ID'] =  data[2]
            bus_route_dic[data[0]]['DMDND_ID'] =  data[3]
            bus_route_dic[data[0]]['stops'] =  data[4:]
            
print bus_route_dic

## Read parking data

In [26]:
input_file_name_parking = os.path.join(output_directory, 'parking_lot')
parking_dic = {} # parking_lot_ID : {'fee($)': int, 'total_capacity': str, 'avg_crusing_time(s)': str, 
                                   # 'connections':[node ID], 'distances':[meters]}
with open(input_file_name_parking, 'r') as infile:
    for line in infile.readlines():
        if line[0] != '#':
            data = line.rstrip('\n').split(' ')
            parking_dic[data[0]] = {}
            parking_dic[data[0]]['fee'] =  int(data[1])
            parking_dic[data[0]]['total_capacity'] =  int(data[2])
            parking_dic[data[0]]['avg_crusing_time'] =  int(data[3])
            parking_dic[data[0]]['connections'] = []
            parking_dic[data[0]]['distances'] = []
            for i in range(4, len(data)):
                parking_dic[data[0]]['connections'].append(data[i].split(':')[0])
                parking_dic[data[0]]['distances'].append(int(data[i].split(':')[1]))
print parking_dic

{'24': {'avg_crusing_time': 100, 'connections': ['1133'], 'fee': 10, 'distances': [100], 'total_capacity': 1000}, '25': {'avg_crusing_time': 100, 'connections': ['1184'], 'fee': 10, 'distances': [100], 'total_capacity': 1000}, '20': {'avg_crusing_time': 100, 'connections': ['1138', '1127'], 'fee': 10, 'distances': [200, 200], 'total_capacity': 1000}, '21': {'avg_crusing_time': 100, 'connections': ['1203'], 'fee': 10, 'distances': [100], 'total_capacity': 1000}, '22': {'avg_crusing_time': 100, 'connections': ['1135', '1147'], 'fee': 10, 'distances': [200, 200], 'total_capacity': 1000}, '23': {'avg_crusing_time': 100, 'connections': ['1213'], 'fee': 10, 'distances': [100], 'total_capacity': 1000}, '1': {'avg_crusing_time': 20, 'connections': ['236'], 'fee': 0, 'distances': [100], 'total_capacity': 2000}, '3': {'avg_crusing_time': 20, 'connections': ['794', '771'], 'fee': 0, 'distances': [200, 200], 'total_capacity': 2000}, '2': {'avg_crusing_time': 20, 'connections': ['214', '759'], 'fee

## Process node

In [35]:
input_file_name_node = os.path.join(input_directory, 'MNM_input_node')
output_file_name_node = os.path.join(output_directory, 'MNM_input_node')
node_str = '# ID Type Convert_factor\n'

with open(input_file_name_node, 'r') as infile:
    for line in infile.readlines():
        if line[0] != '#':
            data = line.rstrip('\r\n').split(' ')
            data.append('2.0')
            node_str += ' '.join(data) + '\n'
    for parking_id in parking_dic:
        new_node_ID = str(90000 + int(parking_id))
        node_str += ' '.join([new_node_ID, 'DMDND', '2.0']) + '\n'
        
f = open(output_file_name_node, 'w')
f.write(node_str)
f.close()

## Process OD

In [25]:
OD_mapping = {} # node_id : OD_id
input_file_name_od = os.path.join(input_directory, 'MNM_input_od')
output_file_name_od = os.path.join(output_directory, 'MNM_input_od')
od_str = ''

with open(input_file_name_od, 'r') as infile:
    for line in infile.readlines():
        od_str += line
        if line[0] != '#':
            data = line.rstrip('\r\n').split(' ')
            OD_mapping[data[1]] = data[0]
            
    for parking_id in parking_dic:
        new_node_ID = str(90000 + int(parking_id))
        od_str += ' '.join([new_node_ID, new_node_ID]) + '\n'
        
f = open(output_file_name_od, 'w')
f.write(od_str)
f.close()

## Process Snap_graph & links

In [38]:
input_file_name_graph = os.path.join(input_directory, 'Snap_graph')
output_file_name_graph = os.path.join(output_directory, 'Snap_graph')
graph_str = ''

input_file_name_link = os.path.join(input_directory, 'MNM_input_link')
output_file_name_link = os.path.join(output_directory, 'MNM_input_link')
link_str = '# ID Type LEN(mile) FFS(mile/h) Cap(v/hour) RHOJ(v/miles) Lane FFS_truck(mile/h) Cap_truck(v/hour) RHOJ_truck(v/miles) Convert_factor(1)\n'

_count_new_link = 0
with open(input_file_name_graph, 'r') as infile:
    with open(input_file_name_link, 'r') as infile2:
        for line in infile.readlines():
            graph_str += line
        for line in infile2.readlines():
            if line[0] != '#':
                data = line.rstrip('\r\n').split(' ')
                if data[1] == 'PQ':
                    data += [data[3], data[4], data[5], '2.0']
                else:
                    data += [str(float(data[3])*0.9), str(float(data[4])*0.9), str(float(data[5])*0.8), '2.0']
                newline = ' '.join(data) + '\n'
                link_str += newline
        
        
        for parking_id in parking_dic:
            to_node_ID = str(90000 + int(parking_id))
            for node in parking_dic[parking_id]['connections']:
                _count_new_link += 1
                link_ID = str(1000000 + _count)
                link_str += ' '.join([link_ID, 'PQ', '1', '99999', '99999', '99999', '1', 
                                      '99999', '99999', '99999', '2.0']) + '\n'
                from_node_ID = node
                graph_str += ' '.join([link_ID, from_node_ID, to_node_ID]) + '\n'
        
f = open(output_file_name_graph, 'w')
f.write(graph_str)
f.close()

f = open(output_file_name_link, 'w')
f.write(link_str)
f.close()

In [39]:
print len(parking_dic)
print _count_new_link

25
43


## Process demand

In [40]:
input_file_name_demand = os.path.join(input_directory, 'MNM_input_demand')
output_file_name_demand = os.path.join(output_directory, 'MNM_input_demand')
demand_str = '# Origin_ID Destination_ID <demand by interval> <truck demand by interval>\n'

with open(input_file_name_demand, 'r') as infile:
    for line in infile.readlines():
        if line[0] != '#':
            demands = line.rstrip('\n').split(' ')
            num_int = len(demands) - 2
            int_sums = [0 for i in range(num_int*2)]
            break
print num_int

with open(input_file_name_demand, 'r') as infile:
    for line in infile.readlines():
        if line[0] != '#':
            demands = line.rstrip('\n').split(' ')
            for i in range(2, 2+num_int):
                demands.append(str(float(demands[i]) * 0.1))
            for i in range(2, 2+num_int):
                demands[i] = str(float(demands[i]) * 0.9)
            demand_str += ' '.join(demands) + '\n'
            
            for i in range(num_int):
                int_sums[i] += float(demands[2 + i])
                int_sums[i + num_int] += float(demands[2 + i + num_int])
                
print int_sums 
print sum(int_sums) 

f = open(output_file_name_demand, 'w')
f.write(demand_str)
f.close()

12
[4259.979539999951, 4259.979539999951, 12779.938619999946, 12779.938619999946, 21299.89770000001, 21299.89770000001, 21299.89770000001, 21299.89770000001, 12779.938619999946, 12779.938619999946, 4259.979539999951, 4259.979539999951, 473.3310600000054, 473.3310600000054, 1419.9931800000268, 1419.9931800000268, 2366.6553000000135, 2366.6553000000135, 2366.6553000000135, 2366.6553000000135, 1419.9931800000268, 1419.9931800000268, 473.3310600000054, 473.3310600000054]
170399.1816
